In [104]:
# ollama list
# ollama pull [model name]
# ollama run [model name]

!pip install ollama
!pip install fsspec

In [126]:
import pandas
import requests
import json
import duckdb

from ollama import chat
from ollama import ChatResponse

import importlib
import columns_to_schema

In [127]:
importlib.reload(columns_to_schema)

<module 'columns_to_schema' from '/Users/sophiawang/Desktop/CS/opendataaccessibility/columns_to_schema.py'>

In [129]:
r = requests.get('https://data.cityofnewyork.us/resource/vx8i-nprf.json')
schema = columns_to_schema.form_schema("data.cityofnewyork.us", "vx8i-nprf", "db")
with open('database.json', 'w') as f:
    json.dump(r.json(), f)

In [143]:
question = 'give me the ids of all candidates with the middle initial A'

prompt = f"""
Generate a SQL query to answer this question: [QUESTION]{question}[/QUESTION]. Do not change the prompt.

Write in plain text without any formatting.

The query will run on a database with the following schema:
{schema}

Answer: Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]. [SQL]
"""

response: ChatResponse = chat(model='sqlcoder:latest', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

output = response['message']['content']
print(output)

<s> SELECT EXAM_NO FROM db WHERE "MI" ILIKE '%A%' ORDER BY exam_no NULLS LAST;


In [139]:
db = duckdb.read_json('database.json')
duckdb.sql(output)

# https://duckdb.org/docs/stable/clients/python/overview.html 

ParserException: Parser Error: syntax error at or near "<"